## STAC Item Properties
A STAC item is a metadata container for spatially and temporally bounded earth observation data. The data can be aerial imagery, radar data or other types of earth observation data. A STAC item has metadata properties describing the dataset and `Assets` that contain information for downloading the data being described. Almost all properties of a STAC item are aspects you can query by using a `StacRequest` with different types of filters.

Return to [README.md](./README.md)

In [1]:
from nsl.stac.client import NSLClient
from nsl.stac import StacRequest

stac_request = StacRequest(id='20190822T183518Z_746_POM1_ST2_P')

# get a client interface to the gRPC channel
client = NSLClient()
# for this request we might as well use the search one, as STAC ids ought to be unique
stac_item = client.search_one(stac_request)

nsl client connecting to stac service at: api.nearspacelabs.net:9090

attempting NSL authentication against https://api.nearspacelabs.net
fetching new authorization in 60 minutes


Here are the sections where we go into more detail about properties and assets.

- [ID, Temporal, and Spatial](#id-temporal-and-spatial)
- [Assets](#assets)
- [Electro Optical](#electro-optical)

Printing out all the data demonstrates what is typically in a StacItem:

In [2]:
print(stac_item)

id: "20190822T183518Z_746_POM1_ST2_P"
collection: "NSL_SCENE"
properties {
  type_url: "nearspacelabs.com/proto/st.protobuf.v1.NslDatast.protobuf.v1.NslData/st.protobuf.v1.NslData"
  value: "\n\340\014\n\03620190822T162258Z_TRAVIS_COUNTY\"\003 \352\0052\03520200702T102306Z_746_ST2_POM1:\03520190822T183518Z_746_POM1_ST2:\03520200702T101632Z_746_ST2_POM1:\03520200702T102302Z_746_ST2_POM1:\03520200702T102306Z_746_ST2_POM1B\03520190822T183518Z_746_POM1_ST2H\001R\374\n\n$\004\304{?\216\371\350=\376\377\306>\300\327\256\275\323rv?2\026*D3Qy6\177>\3675\000\000\200?\022\024\r+}\303\302\025\033;\362A\0353}\367\300%g\232\250@\022\024\r\026}\303\302\025\376?\362A\035\000\367\235@%\232\t\331?\022\024\r\351|\303\302\025\021A\362A\035M\370\033\301%g\016\226\277\022\024\r\201|\303\302\025\3709\362A\035\000\252\245@%\315\3547?\022\024\r\310|\303\302\025\245G\362A\035\232\315l\301%3\347\270\300\022\024\rq|\303\302\025\2149\362A\035\000\376o@%\000(\017@\022\024\rD|\303\302\025oD\362A\0353\323\302\301%\3

In addition to spatial and temporal details there are also details about the capturing device. We use both strings (to stay compliant with STAC JSON) and enum fields for these details. The `platform_enum` and `platform` is the model of the vehicle holding the sensor. The `instrument_enum` and `instrument` is the sensor that collected the scenes. In our case we're using `mission_enum` and `mission` to represent a class of flight vehicles that we're flying. In the case of the Landsat satellite program the breakdown would be:

 *   `platform_enum`: `enum.PLATFORM.LANDSAT_8`
 *   `sensor_enum`: `enum.SENSOR.OLI_TIRS`
 *   `mission_enum`: `enum.MISSION.LANDSAT`
 *   `platform`: "LANDSAT_8"
 *   `sensor`: "OLI_TIRS"
 *   `mission`: "LANDSAT"


### ID Temporal and Spatial
Every STAC Item has a unique id, a datetime/observation, and a geometry/bbox (bounding-box).

In [3]:
print("STAC Item id: {}\n".format(stac_item.id))
print("STAC Item observed: {}".format(stac_item.observed))
print("STAC Item datetime: {}".format(stac_item.datetime))
print("STAC Item bbox: {}".format(stac_item.bbox))
print("STAC Item geometry: {}".format(stac_item.geometry))

STAC Item id: 20190822T183518Z_746_POM1_ST2_P

STAC Item observed: seconds: 1566498918
nanos: 505476000

STAC Item datetime: seconds: 1566498918
nanos: 505476000

STAC Item bbox: xmin: -97.7466867683867
ymin: 30.278398961994966
xmax: -97.72990596574927
ymax: 30.288621181865743
proj {
  epsg: 4326
}

STAC Item geometry: wkb: "\001\006\000\000\000\001\000\000\000\001\003\000\000\000\001\000\000\000\005\000\000\000\352\244L\267\311oX\300\316\340\320\247\234I>@\241\273\2606\267oX\300<\002\205\'EG>@\031\003\203\307\266nX\3001z\244\372\233G>@CCAI\306nX\300\326\013\351\023\343I>@\352\244L\267\311oX\300\316\340\320\247\234I>@"
proj {
  epsg: 4326
}
envelope {
  xmin: -97.7466867683867
  ymin: 30.278398961994966
  xmax: -97.72990596574927
  ymax: 30.288621181865743
  proj {
    epsg: 4326
  }
}
simple: STRONG_SIMPLE



As you can see above, the `id` is a string value. The format of the id is typically not guessable (ours is based of off the time the data was processed, the image index, the platform and the sensor).

The `observed` and `datetime` fields are the same value. STAC specification uses a generic field `datetime` to define the spatial component, the `S`, in STAC. We wanted a more descriptive variable, so we use `observed`, as in, the moment the scene was captured. This is a UTC timestamp in seconds and nano seconds.

The `bbox` field describes the xmin, ymin, xmax, and ymax points that describe the bounding box that contains the scene. The `sr` field has an [epsg](http://www.epsg.org/) `wkid`. In this case the 4326 `wkid` indicates [WGS-84](http://epsg.io/4326)

The `geometry` field has subfields `wkb`, `sr`, and `simple`. The `wkb` is a [well known binary](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry#Well-known_binary) geometry format preferred for it's size. `sr` is the same as in the `bbox`. `simple` can be ignored.

Below we demonstrate how you can create python `datetime` objects:

In [4]:
from datetime import datetime
print("UTC Observed Scene: {}".format(datetime.utcfromtimestamp(stac_item.observed.seconds)))
print("UTC Processed Data: {}".format(datetime.utcfromtimestamp(stac_item.created.seconds)))
print("UTC Updated Metadata: {}".format(datetime.utcfromtimestamp(stac_item.updated.seconds)))

UTC Observed Scene: 2019-08-22 18:35:18
UTC Processed Data: 2020-08-06 19:56:51
UTC Updated Metadata: 2021-02-01 15:28:06


Updated is when the metadata was last updated. Typically that will be right after it's `processed` timestamp.

Below is a demo of using shapely to get at the geometry data.

In [5]:
from shapely.geometry import Polygon
from shapely.wkb import loads

print("wkt printout of polygon:\n{}\n".format(loads(stac_item.geometry.wkb)))
print("centroid of polygon:\n{}\n".format(loads(stac_item.geometry.wkb).centroid))
print("bounds:\n{}\n".format(Polygon.from_bounds(stac_item.bbox.xmin, 
                                                 stac_item.bbox.ymin, 
                                                 stac_item.bbox.xmax, 
                                                 stac_item.bbox.ymax)))

wkt printout of polygon:
MULTIPOLYGON (((-97.7466867683867 30.28754662370266, -97.74555747279238 30.27839896199497, -97.72990596574927 30.27972380176124, -97.73085242627444 30.28862118186574, -97.7466867683867 30.28754662370266)))

centroid of polygon:
POINT (-97.738289581264 30.28357703330576)

bounds:
POLYGON ((-97.7466867683867 30.27839896199497, -97.7466867683867 30.28862118186574, -97.72990596574927 30.28862118186574, -97.72990596574927 30.27839896199497, -97.7466867683867 30.27839896199497))



### Assets
Each STAC item should have at least one asset. An asset should be all the information you'll need to download the asset in question. For Near Space Labs customers, you'll be using the href, but you can also see the private bucket details of the asset. In protobuf the asset map has a key for each asset available. There's no part of the STAC specification for defining key names. Near Space Labs typically uses the data type, the optical bands and the cloud storage provider to construct a key name.

In [6]:
from nsl.stac import Asset, utils
from nsl.stac.enum import AssetType
def print_asset(asset: Asset):
    asset_name = AssetType(asset.asset_type).name
    print(" href: {}".format(asset.href))
    print(" type: {}".format(asset.type))
    print(" protobuf enum number and name: {0}, {1}".format(asset.asset_type, asset_name))
    print()

print("there are {} assets".format(len(stac_item.assets)))
print(AssetType.THUMBNAIL.name)
print_asset(utils.get_asset(stac_item, asset_type=AssetType.THUMBNAIL))

print(AssetType.GEOTIFF.name)
print_asset(utils.get_asset(stac_item, asset_type=AssetType.GEOTIFF))

there are 2 assets
THUMBNAIL
 href: https://api.nearspacelabs.net/download/20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183518Z_746_POM1_ST2_P.png
 type: image/png
 protobuf enum number and name: 9, THUMBNAIL

GEOTIFF
 href: https://api.nearspacelabs.net/download/20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183518Z_746_POM1_ST2_P.tif
 type: image/vnd.stac.geotiff
 protobuf enum number and name: 2, GEOTIFF



As you can see above, our data only consists of jpg thumbnails and Geotiffs. But there can be other data stored in Assets in the future.

You can read more details about Assets [here](https://geo-grpc.github.io/api/#epl.protobuf.Asset)

### View
Some imagery analysis tools require knowing certain types of angular information. Here's a printout of the information we've collected with data. A summary of View values can be found [here](https://geo-grpc.github.io/api/#epl.protobuf.v1.View).

In [7]:
print(stac_item.view)

off_nadir {
  value: 9.42326831817627
}
azimuth {
  value: -74.85270690917969
}
sun_azimuth {
  value: 181.26959228515625
}
sun_elevation {
  value: 71.41288757324219
}



These `sun_azimuth`, `sun_elevation`, `off_nadir` and `azimuth` are all boxed in the [google.protobuf.FloatValue type](https://developers.google.com/protocol-buffers/docs/reference/csharp/class/google/protobuf/well-known-types/float-value). To get at the value you must access the `value` field.

In [8]:
print("sun_azimuth: {:.5f}".format(stac_item.view.sun_azimuth.value))
print("sun_elevation: {:.5f}".format(stac_item.view.sun_elevation.value))
print("off_nadir: {:.5f}".format(stac_item.view.off_nadir.value))
print("azimuth: {:.5f}".format(stac_item.view.azimuth.value))

sun_azimuth: 181.26959
sun_elevation: 71.41289
off_nadir: 9.42327
azimuth: -74.85271


Notice that we're only printing out 5 decimal places. As these are stored as float values, we can't trust any of the precision that Python provides us beyond what we know the data to possess.

You can read more details about electro-optical data [here](https://geo-grpc.github.io/api/#epl.protobuf.Eo)

Return to [README.md](./README.md)